# 네이버 뉴스 크롤링 - Graph DB 데이터 수집

네이버 뉴스에서 분야별 헤드라인 기사 수집

- **정치**: https://news.naver.com/section/100
- **경제**: https://news.naver.com/section/101
- **사회**: https://news.naver.com/section/102
- **생활/문화**: https://news.naver.com/section/103
- **IT/과학**: https://news.naver.com/section/105
- **세계**: https://news.naver.com/section/104

## 수집 항목 (Articles.xlsx)
- article_id: 기사 고유 ID
- title: 기사 제목
- content: 기사 본문
- url: 기사 URL
- published_date: 발행일
- source: 언론사
- author: 기자명
- category: 카테고리

## 1. 필요한 패키지 설치 및 임포트

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from datetime import datetime
import re

## 2. 카테고리 설정 및 Selenium 초기화

In [2]:
categories = {
    '정치': 'https://news.naver.com/section/100',
    '경제': 'https://news.naver.com/section/101',
    '사회': 'https://news.naver.com/section/102',
    '생활/문화': 'https://news.naver.com/section/103',
    'IT/과학': 'https://news.naver.com/section/105',
    '세계': 'https://news.naver.com/section/104'
}

NUM_ARTICLES_PER_CATEGORY = 10

In [3]:
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=service, options=options)

## 3. 기사 수집 함수 정의

In [4]:
def get_article_links(driver, category_url, num_articles):
    driver.get(category_url)
    time.sleep(3)

    article_links = []

    try:
        selectors = [
            'a.sa_text_lede',
            'a.sa_text_strong',
            '.sa_text a',
            '.cluster_text_headline a',
            '.cluster_text_lede a'
        ]

        for selector in selectors:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                url = element.get_attribute('href')
                if (url and 'news.naver.com' in url and '/article/' in url
                    and '/comment/' not in url  # 댓글 페이지만 제외
                    and url not in article_links):
                    article_links.append(url)
                    if len(article_links) >= num_articles:
                        break
            if len(article_links) >= num_articles:
                break

        print(f"✓ {len(article_links)}개의 기사 링크 수집 완료")

    except Exception as e:
        print(f"✗ 기사 링크 수집 실패: {e}")

    return article_links[:num_articles]

In [5]:
def parse_article_detail(driver, article_url, category):
    driver.get(article_url)
    time.sleep(1.5)

    article_data = {
        'article_id': '',
        'title': '',
        'content': '',
        'url': article_url,
        'published_date': '',
        'source': '',
        'author': '',
        'category': category
    }

    try:
        # 기사 ID 생성 (URL에서 추출)
        match = re.search(r'article/(\d+)/(\d+)', article_url)
        if match:
            article_data['article_id'] = f"ART_{match.group(1)}_{match.group(2)}"
        else:
            article_data['article_id'] = f"ART_{datetime.now().strftime('%Y%m%d%H%M%S')}"

        # 제목
        title_selectors = [
            '#title_area span',
            '#ct .media_end_head_headline',
            '.media_end_head_headline',
            'h2#title_area',
            '.news_end_title'
        ]

        for selector in title_selectors:
            try:
                title_element = driver.find_element(By.CSS_SELECTOR, selector)
                if title_element.text.strip():
                    article_data['title'] = title_element.text.strip()
                    break
            except:
                continue

        # 본문
        content_selectors = [
            '#dic_area',
            'article#dic_area',
            '.go_trans._article_content',
            '._article_body_contents'
        ]

        for selector in content_selectors:
            try:
                content_element = driver.find_element(By.CSS_SELECTOR, selector)
                if content_element.text.strip():
                    article_data['content'] = content_element.text.strip()
                    break
            except:
                continue

        # 언론사
        try:
            source_element = driver.find_element(By.CSS_SELECTOR, 'a.media_end_head_top_logo img')
            article_data['source'] = source_element.get_attribute('alt')
        except:
            try:
                source_element = driver.find_element(By.CSS_SELECTOR, '.media_end_head_top_logo_text')
                article_data['source'] = source_element.text.strip()
            except:
                pass

        # 발행일
        try:
            date_element = driver.find_element(By.CSS_SELECTOR, 'span.media_end_head_info_datestamp_time, span[data-date-time]')
            date_text = date_element.get_attribute('data-date-time') or date_element.text
            article_data['published_date'] = date_text.strip()
        except:
            article_data['published_date'] = datetime.now().strftime('%Y-%m-%d %H:%M')

        # 기자명
        try:
            author_element = driver.find_element(By.CSS_SELECTOR, 'em.media_end_head_journalist_name, span.byline_s')
            article_data['author'] = author_element.text.strip()
        except:
            pass

    except Exception as e:
        print(f"  ✗ 파싱 오류: {e}")

    return article_data

## 4. 전체 카테고리 크롤링 실행

In [6]:
# 전체 기사 데이터를 저장할 리스트
all_articles = []

# 카테고리별 크롤링
for category_name, category_url in categories.items():
    print(f"\n{'='*60}")
    print(f"[{category_name}] 카테고리 수집 시작...")
    print(f"{'='*60}")

    # 1단계: 기사 링크 수집
    article_links = get_article_links(driver, category_url, NUM_ARTICLES_PER_CATEGORY)

    # 2단계: 각 기사 상세 정보 수집
    for idx, article_url in enumerate(article_links, 1):
        print(f"  [{idx}/{len(article_links)}] {article_url}")
        article_data = parse_article_detail(driver, article_url, category_name)

        if article_data['title']:  # 제목이 있는 경우만 추가
            all_articles.append(article_data)
            print(f"  ✓ 수집 완료: {article_data['title'][:50]}...")
        else:
            print(f"  ✗ 수집 실패 - 제목을 찾을 수 없습니다.")

        time.sleep(0.5)


[정치] 카테고리 수집 시작...
✓ 10개의 기사 링크 수집 완료
  [1/10] https://n.news.naver.com/mnews/article/018/0006168585
  ✓ 수집 완료: 北, 사업 성공한 부부 ‘공개 처형’…“태도 오만해”...
  [2/10] https://n.news.naver.com/mnews/article/421/0008616962
  ✓ 수집 완료: 李대통령, 남아공서 프랑스·독일과 정상회담…"국제정세 변화 대응"...
  [3/10] https://n.news.naver.com/mnews/article/366/0001124550
  ✓ 수집 완료: ‘마이웨이’ 법사위에 뿔난 與 지도부… “대통령 외교 성과 또 빛바랠라’”...
  [4/10] https://n.news.naver.com/mnews/article/052/0002276132
  ✓ 수집 완료: 이 대통령 "남북 대결 끝내고 평화 공존의 시대로...단계적 비핵화 추진"...
  [5/10] https://n.news.naver.com/mnews/article/449/0000327216
  ✓ 수집 완료: 이 대통령 지지율 60%…민주 43%·국힘 24%...
  [6/10] https://n.news.naver.com/mnews/article/277/0005682723
  ✓ 수집 완료: 송언석 "정부혁신 TF, 야만적 공무원 줄세우기...김문기법 추진"...
  [7/10] https://n.news.naver.com/mnews/article/366/0001124755
  ✓ 수집 완료: 李대통령, 이집트 정상회담 종료… “국제 평화 함께 기여… 방산 협력 확대”...
  [8/10] https://n.news.naver.com/mnews/article/079/0004088251
  ✓ 수집 완료: 金여사, 이집트 영부인과 친교…"'K-할랄푸드' 대접하고파"...
  [9/10] https://n.news.naver.com/mnews/article/015

## 5. 수집 결과 확인

In [7]:
df_articles = pd.DataFrame(all_articles)

In [8]:
df_articles

,article_id,title,content,url,published_date,source,author,category
0,ART_018_0006168585,"北, 사업 성공한 부부 ‘공개 처형’…“태도 오만해”","북한 개인 사업 운영 50대 부부,\n“오만해졌다” 야외 공간서 처형\n당국 “대중...",https://n.news.naver.com/mnews/article/018/000...,2025-11-20 14:42:17,이데일리,권혜미 기자,정치
1,ART_421_0008616962,"李대통령, 남아공서 프랑스·독일과 정상회담…""국제정세 변화 대응""","G20 계기 유럽 핵심 강국 佛·獨 양자회담 성사\n""MIKTA 회동서 다자주의·국...",https://n.news.naver.com/mnews/article/421/000...,2025-11-21 03:37:20,뉴스1,심언기 기자,정치
2,ART_366_0001124550,‘마이웨이’ 법사위에 뿔난 與 지도부… “대통령 외교 성과 또 빛바랠라’”,김병기 “협의했어야… 뒷감당은 법사위가”\n\n더불어민주당 지도부와 국회 법제사법위...,https://n.news.naver.com/mnews/article/366/000...,2025-11-20 11:55:09,조선비즈,박숙현 기자,정치
3,ART_052_0002276132,"이 대통령 ""남북 대결 끝내고 평화 공존의 시대로...단계적 비핵화 추진""","이집트를 공식 방문 중인 이재명 대통령은 남북 적대와 대결의 시대를 끝내고, 평화 ...",https://n.news.naver.com/mnews/article/052/000...,2025-11-21 03:06:56,YTN,홍민기 기자,정치
4,ART_449_0000327216,이 대통령 지지율 60%…민주 43%·국힘 24%,이집트를 공식 방문 중인 이재명 대통령과 김혜경 여사가 20일(현지 시간) 카이로 ...,https://n.news.naver.com/mnews/article/449/000...,2025-11-21 10:52:16,채널A,윤승옥 기자 touch@ichannela.com,정치
5,ART_277_0005682723,"송언석 ""정부혁신 TF, 야만적 공무원 줄세우기...김문기법 추진""","""공무원 성실 행정 면책 입법 추진""\n송언석 국민의힘 원내대표는 21일 공무원을 ...",https://n.news.naver.com/mnews/article/277/000...,2025-11-21 10:18:33,아시아경제,최유리 기자,정치
6,ART_366_0001124755,"李대통령, 이집트 정상회담 종료… “국제 평화 함께 기여… 방산 협력 확대”",이집트를 공식 방문 중인 이재명 대통령은 20일(현지시각) 압델 파타 알시시 대통령...,https://n.news.naver.com/mnews/article/366/000...,2025-11-20 21:45:10,조선비즈,이윤정 기자,정치
7,ART_079_0004088251,"金여사, 이집트 영부인과 친교…""'K-할랄푸드' 대접하고파""","핵심요약\n인티사르 여사 ""예전처럼 알던 친구같다…교류 활발해지기를""\n이재명 대통...",https://n.news.naver.com/mnews/article/079/000...,2025-11-21 01:33:08,노컷뉴스,이준규 기자,정치
8,ART_015_0005214392,"李대통령 ""민주주의의 우뚝 선 봉우리""…김영삼 전 대통령 추모",사진=연합뉴스\n\n이재명 대통령은 김영삼 전 대통령 서거 10주기인 21일 김 전...,https://n.news.naver.com/mnews/article/015/000...,2025-11-21 11:07:11,한국경제,박수림 기자,정치
9,ART_016_0002561105,‘공천 혁명’이냐 ‘연임 노림수’냐…정청래 ‘1인1표제’ 실현 임박[이런정치],더불어민주당 당헌·당규 개정 의결 과정 착수\n정청래 “90% 당원 뜻은 거스를 수...,https://n.news.naver.com/mnews/article/016/000...,2025-11-21 10:31:11,헤럴드경제,양근혁 기자,정치


## 6. Excel 파일로 저장

In [9]:
output_filename = f"Articles_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
df_articles.to_excel(output_filename, index=False, engine='openpyxl')

In [10]:
driver.quit()